In [11]:
import requests

# API endpoint for Realtor v3 list
url = "https://realty-in-us.p.rapidapi.com/properties/v3/list"

# Headers with your API key and host
headers = {
    "content-type": "application/json",
    "X-RapidAPI-Key": "33f7baa8aemshb3380eca00fc1c0p16a229jsn6f972fd9d47e",  # <-- Replace if needed
    "X-RapidAPI-Host": "realty-in-us.p.rapidapi.com"
}

# Request payload (search by ZIP code in Phoenix)
payload = {
    "limit": 20,
    "offset": 0,
    "postal_code": "85004",  # You can replace with any Phoenix ZIP
    "status": ["for_sale"],
    "sort": {
        "direction": "desc",
        "field": "list_date"
    }
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)

# Parse the response
if response.status_code == 200:
    data = response.json()
    listings = data.get("data", {}).get("home_search", {}).get("results", [])
    
    if not listings:
        print("No listings found. Try another ZIP or adjust filters.")
    else:
        for i, home in enumerate(listings, 1):
            address = home['location']['address'].get('line', 'N/A')
            city = home['location']['address'].get('city', 'N/A')
            zipcode = home['location']['address'].get('postal_code', 'N/A')
            price = home.get('list_price', 'N/A')
            lat = home['location']['address'].get('coordinate', {}).get('lat', 'N/A')
            lon = home['location']['address'].get('coordinate', {}).get('lon', 'N/A')
            baths = home.get('description', {}).get('baths', 'N/A')
            beds = home.get('description', {}).get('beds', 'N/A')
            lot_size = home.get('description', {}).get('lot_sqft', 'N/A')
            type_ = home.get('description', {}).get('type', 'N/A')
            url = home.get('href', 'N/A')
     
            
            # Print nicely
            print(f"{i}. {address}, {city} {zipcode}")
            print(f"   Price: ${price:,} | Beds: {beds} | Baths: {baths}")
            print(f"   Coordinates: ({lat}, {lon})")
            print(f"   URL: {url}")
            print("-" * 60)

else:
    print(f"Error {response.status_code}: {response.text}")

1. 424 S 2nd St Unit 203, Phoenix 85004
   Price: $350,000 | Beds: 1 | Baths: 2
   Coordinates: (33.44348, -112.071442)
   URL: https://www.realtor.com/realestateandhomes-detail/424-S-2nd-St-Unit-203_Phoenix_AZ_85004_M21292-79156
------------------------------------------------------------
2. 1700 S 1st St Lot 1, Phoenix 85004
   Price: $275,000 | Beds: None | Baths: 0
   Coordinates: (33.431198, -112.07296)
   URL: https://www.realtor.com/realestateandhomes-detail/1700-S-1st-St-1_Phoenix_AZ_85004_M92311-69311
------------------------------------------------------------
3. 377 E Windsor Ave Unit 24, Phoenix 85004
   Price: $750,000 | Beds: 2 | Baths: 3
   Coordinates: (33.477791, -112.065941)
   URL: https://www.realtor.com/realestateandhomes-detail/377-E-Windsor-Ave-Unit-24_Phoenix_AZ_85004_M92796-72545
------------------------------------------------------------
4. 361 E Palm Ln, Phoenix 85004
   Price: $595,000 | Beds: 2 | Baths: 2
   Coordinates: (33.468637, -112.066794)
   URL: ht

In [13]:
import pandas as pd

house_data = []

for home in listings:
    address = home['location']['address'].get('line', 'N/A')
    city = home['location']['address'].get('city', 'N/A')
    zipcode = home['location']['address'].get('postal_code', 'N/A')
    price = home.get('list_price', None)
    lat = home['location']['address'].get('coordinate', {}).get('lat', None)
    lon = home['location']['address'].get('coordinate', {}).get('lon', None)
    baths = home.get('description', {}).get('baths', None)
    beds = home.get('description', {}).get('beds', None)
    lot_size = home.get('description', {}).get('lot_sqft', None)
    type_ = home.get('description', {}).get('type', None)
    url = home.get('href', 'N/A')

    if lat is not None and lon is not None:
        house_data.append({
            "address": address,
            "city": city,
            "zipcode": zipcode,
            "price": price,
            "lat": lat,
            "lon": lon,
            "beds": beds,
            "baths": baths,
            "lot_size": lot_size,
            "type": type_,
            "url": url
        })

houses_df = pd.DataFrame(house_data)

In [15]:
houses_df

,address,city,zipcode,price,lat,lon,beds,baths,lot_size,type,url
0,424 S 2nd St Unit 203,Phoenix,85004,350000,33.443480,-112.071442,1.0,2.0,119.0,condos,https://www.realtor.com/realestateandhomes-det...
1,1700 S 1st St Lot 1,Phoenix,85004,275000,33.431198,-112.072960,NaN,0.0,5480.0,land,https://www.realtor.com/realestateandhomes-det...
2,377 E Windsor Ave Unit 24,Phoenix,85004,750000,33.477791,-112.065941,2.0,3.0,1460.0,townhomes,https://www.realtor.com/realestateandhomes-det...
3,361 E Palm Ln,Phoenix,85004,595000,33.468637,-112.066794,2.0,2.0,2509.0,single_family,https://www.realtor.com/realestateandhomes-det...
4,310 S 4th St Unit 601,Phoenix,85004,538900,33.444553,-112.069165,2.0,3.0,1625.0,condos,https://www.realtor.com/realestateandhomes-det...
5,312 E Windsor Ave Unit 6,Phoenix,85004,879000,33.478319,-112.068805,3.0,3.0,922.0,townhomes,https://www.realtor.com/realestateandhomes-det...
6,1130 N 2nd St Apt 306,Phoenix,85004,317000,33.461033,-112.071567,1.0,1.0,490.0,condos,https://www.realtor.com/realestateandhomes-det...
7,1014 S 2nd St,Phoenix,85004,545000,33.437522,-112.071553,NaN,NaN,7415.0,multi_family,https://www.realtor.com/realestateandhomes-det...
8,1130 N 2nd St Apt 208,Phoenix,85004,454000,33.460956,-112.071565,2.0,2.0,984.0,condos,https://www.realtor.com/realestateandhomes-det...
9,325 E Coronado Rd Unit 8,Phoenix,85004,630000,33.467175,-112.068093,2.0,3.0,940.0,townhomes,https://www.realtor.com/realestateandhomes-det...


In [20]:
risk_df = pd.read_csv(r"C:\Users\tomas\Capstone\phoenix_scores.csv")

In [22]:
from scipy.spatial import cKDTree

# Prepare coordinates
risk_coords = risk_df[["lat", "lon"]].drop_duplicates().values
tree = cKDTree(risk_coords)

# Find nearest risk coordinate for each house
_, idxs = tree.query(houses_df[["lat", "lon"]].values)
nearest_risk_coords = risk_coords[idxs]

# Merge coordinates into house dataframe
houses_df["nearest_lat"] = nearest_risk_coords[:, 0]
houses_df["nearest_lon"] = nearest_risk_coords[:, 1]


In [30]:
houses_df = houses_df.merge(
    risk_df.rename(columns={'lat': 'nearest_lat', 'lon': 'nearest_lon'}),
    on=['nearest_lat', 'nearest_lon'],
    how='left'
)

In [32]:
houses_df

,address,city,zipcode,price,lat,lon,beds,baths,lot_size,type,url,nearest_lat,nearest_lon,Unnamed: 0,std_score
0,424 S 2nd St Unit 203,Phoenix,85004,350000,33.443480,-112.071442,1.0,2.0,119.0,condos,https://www.realtor.com/realestateandhomes-det...,33.441667,-112.058333,46,0.306647
1,1700 S 1st St Lot 1,Phoenix,85004,275000,33.431198,-112.072960,NaN,0.0,5480.0,land,https://www.realtor.com/realestateandhomes-det...,33.441667,-112.058333,46,0.306647
2,377 E Windsor Ave Unit 24,Phoenix,85004,750000,33.477791,-112.065941,2.0,3.0,1460.0,townhomes,https://www.realtor.com/realestateandhomes-det...,33.483333,-112.058333,56,0.498162
3,361 E Palm Ln,Phoenix,85004,595000,33.468637,-112.066794,2.0,2.0,2509.0,single_family,https://www.realtor.com/realestateandhomes-det...,33.483333,-112.058333,56,0.498162
4,310 S 4th St Unit 601,Phoenix,85004,538900,33.444553,-112.069165,2.0,3.0,1625.0,condos,https://www.realtor.com/realestateandhomes-det...,33.441667,-112.058333,46,0.306647
5,312 E Windsor Ave Unit 6,Phoenix,85004,879000,33.478319,-112.068805,3.0,3.0,922.0,townhomes,https://www.realtor.com/realestateandhomes-det...,33.483333,-112.058333,56,0.498162
6,1130 N 2nd St Apt 306,Phoenix,85004,317000,33.461033,-112.071567,1.0,1.0,490.0,condos,https://www.realtor.com/realestateandhomes-det...,33.441667,-112.058333,46,0.306647
7,1014 S 2nd St,Phoenix,85004,545000,33.437522,-112.071553,NaN,NaN,7415.0,multi_family,https://www.realtor.com/realestateandhomes-det...,33.441667,-112.058333,46,0.306647
8,1130 N 2nd St Apt 208,Phoenix,85004,454000,33.460956,-112.071565,2.0,2.0,984.0,condos,https://www.realtor.com/realestateandhomes-det...,33.441667,-112.058333,46,0.306647
9,325 E Coronado Rd Unit 8,Phoenix,85004,630000,33.467175,-112.068093,2.0,3.0,940.0,townhomes,https://www.realtor.com/realestateandhomes-det...,33.483333,-112.058333,56,0.498162


In [45]:
import pandas as pd

# Parameters based on the report
ALPHA = 0.5   # Premium coefficient for CRS ≤ 0.2
BETA = 0.4    # Penalty coefficient for CRS > 0.2
THRESHOLD = 0.2  # CRS threshold

# Define calculation logic
def calculate_capi(row, alpha=ALPHA, beta=BETA, threshold=THRESHOLD):
    crs = row['std_score']
    market_price = row['price']
    
    if crs <= threshold:
        uplift = alpha * (threshold - crs)
        capi_price = market_price * (1 + uplift)
    else:
        penalty = beta * (crs - threshold) ** 2
        capi_price = market_price * (1 - penalty)
    
    return capi_price

# Apply the calculation
houses_df['capi_price'] = houses_df.apply(calculate_capi, axis=1)
houses_df['adjustment_%'] = ((houses_df['capi_price'] - houses_df['price']) / houses_df['price'] * 100).round(2)
houses_df['capi_price'] = houses_df['capi_price'].apply(lambda x: f"{x:.2f}")

# Display the result
houses_df[['address', 'price', 'std_score', 'capi_price', 'adjustment_%']]

,address,price,std_score,capi_price,adjustment_%
0,424 S 2nd St Unit 203,350000,0.306647,348407.69,-0.45
1,1700 S 1st St Lot 1,275000,0.306647,273748.90,-0.45
2,377 E Windsor Ave Unit 24,750000,0.498162,723329.79,-3.56
3,361 E Palm Ln,595000,0.498162,573841.63,-3.56
4,310 S 4th St Unit 601,538900,0.306647,536448.30,-0.45
5,312 E Windsor Ave Unit 6,879000,0.498162,847742.51,-3.56
6,1130 N 2nd St Apt 306,317000,0.306647,315557.82,-0.45
7,1014 S 2nd St,545000,0.306647,542520.55,-0.45
8,1130 N 2nd St Apt 208,454000,0.306647,451934.55,-0.45
9,325 E Coronado Rd Unit 8,630000,0.498162,607597.02,-3.56
